### colab으로 할 때만 실행

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
!nvidia-smi

Thu Dec 15 06:52:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   75C    P0    34W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import os
os.chdir("/content/drive/MyDrive")

### 설치 필요할때만

In [ ]:
!python --version
!pip install opencv-python
!pip install tensorflow_datasets
!pip install -U scikit-learn
!pip install adabelief-tf
!pip install scipy

### 기본 설정

In [4]:
from shutil import copy
from collections import defaultdict
import scipy
import numpy as np
import matplotlib.pyplot as plt

import random
import cv2
import glob
from PIL import Image, ImageEnhance
import PIL.ImageOps

import torch
import torchvision
from torch import nn

import warnings

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

import tensorflow as tf
from tensorflow import keras
from tensorflow.python.ops.clip_ops import clip_by_global_norm
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers
import tensorflow_datasets as tfds


# Theano(th)와 Tensorflow(tf) 모두와 호환이 되는 Keras 모듈을 작성
import keras.backend as K
from keras import regularizers
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation, BatchNormalization, Dropout, Flatten, Dense, concatenate
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator # 데이터 전처리
from keras.callbacks import ModelCheckpoint, CSVLogger
#from keras.optimizers import SGD

from keras_preprocessing.image import array_to_img, img_to_array, load_img

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold


from deeplab2.model.pixel_encoder import moat
from adabelief_tf import AdaBeliefOptimizer

### 데이터 불러오기

### https://www.tensorflow.org/datasets/catalog/cifar100

In [5]:
# 'cifar100' 대신 'food101' 이나 다른 데이터셋 이름 넣으면 ok
from tensorflow.keras.datasets import cifar100

(train_images, train_labels), (test_images, test_labels) = cifar100.load_data()

tr1_images, tr2_images, tr1_labels, tr2_labels = train_test_split(train_images, train_labels, test_size=0.5, shuffle=False)

nb_train_samples =  train_images.shape[0]
nb_train1_samples =  tr1_images.shape[0]
nb_train2_samples =  tr2_images.shape[0] 
nb_test_samples = test_images.shape[0]
batch_size = 32

print('Train:', train_images.shape, train_labels.shape)
print('Train1:', tr1_images.shape, tr1_labels.shape)
print('Train2:', tr2_images.shape, tr2_labels.shape)
print('Test:', test_images.shape, test_labels.shape)

#print(np.unique(tr_labels, return_counts = True))
#print(np.unique(tr1_labels, return_counts = True))
#print(np.unique(tr2_labels, return_counts = True))
#print(np.unique(test_labels, return_counts = True))

Train: (50000, 32, 32, 3) (50000, 1)
Train1: (25000, 32, 32, 3) (25000, 1)
Train2: (25000, 32, 32, 3) (25000, 1)
Test: (10000, 32, 32, 3) (10000, 1)


### ImageDataGenerator

In [6]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,        # Shear angle in counter-clockwise direction as radians
    zoom_range=0.3,         # Range for random zoom. If a float
    horizontal_flip=True,   # Randomly flip inputs horizontally
    fill_mode='nearest') 

test_datagen = ImageDataGenerator(
    rescale=1./255,)

### 모델 생성

In [7]:
moat1 = moat.get_model(name='tiny_moat1_pretrain_256_no_pe_1k',input_shape=[32,32,3])
moat1 = moat._load_moat_pretrained_checkpoint(moat1, path="./model-ckpt-0")

o1 = moat1.output["stage5"]
r1 = tf.keras.layers.Flatten(name='flatten_layer1')(o1)
r1 = tf.keras.layers.BatchNormalization()(r1)
out1 = tf.keras.layers.Dense(
    units=100, activation='softmax', name='output_layer', kernel_initializer="he_normal")(r1)  

def create_model1():
    return tf.keras.models.Model(moat1.input, out1)

In [8]:
moat2 = moat.get_model(name='tiny_moat1_pretrain_256_no_pe_1k',input_shape=[32,32,3])
moat2 = moat._load_moat_pretrained_checkpoint(moat2, path="./model-ckpt-0")

o2 = moat2.output["stage5"]
r2 = tf.keras.layers.Flatten(name='flatten_layer1')(o2)
r2 = tf.keras.layers.BatchNormalization()(r2)
out2 = tf.keras.layers.Dense(
    units=100, activation='softmax', name='output_layer', kernel_initializer="he_normal")(r2)  

def create_model2():
    return tf.keras.models.Model(moat2.input, out2)

In [ ]:
warnings.filterwarnings('ignore')

skf = StratifiedKFold(n_splits = 5)

for train_index, test_index in skf.split(tr1_images, tr1_labels):
    
    tr_images, val_images = tr1_images[train_index], tr1_images[test_index]
    tr_labels, val_labels = tr1_labels[train_index], tr1_labels[test_index]
    
    nb_tr_samples =  tr_images.shape[0]
    nb_val_samples =  val_images.shape[0]
    
    train_generator = train_datagen.flow(x=tr_images, y=tr_labels, batch_size=100, shuffle=True)
    validation_generator = test_datagen.flow(x=val_images, y=val_labels, batch_size=100, shuffle=False)

    model1 = create_model1()
    
    try:
        model1.load_weights('cifar_final_disjoint_kfold_1_1.hdf5')
    except:
        pass
        

    opt = AdaBeliefOptimizer(learning_rate=1e-3, beta_1= 0.9, beta_2=0.999, weight_decay= 0.05,  epsilon=1e-14)

    model1.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())

    checkpointer = ModelCheckpoint(filepath='cifar_final_disjoint_kfold_1_1.hdf5', verbose=1, monitor ="val_sparse_categorical_accuracy",save_best_only=True)
    csv_logger = CSVLogger('cifar_final_disjoint_kfold_1_1.log')

    history = model1.fit(train_generator,
                    steps_per_epoch = nb_tr_samples // 100,      
                    validation_data=validation_generator,
                    validation_steps=nb_val_samples // 100,  
                    epochs=10,                                             
                    verbose=1,
                    callbacks=[csv_logger, checkpointer])

    

Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
>=0.1.0 (Current 0.2.1)  1e-14  supported          default: True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended epsilon = 1e-7                                  Recommended epsilon = 1e-14
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.

Epoch 1/10
200/200 [==============================] - ETA: 0s - loss: 3.6711 - sparse_categorical

### 데이터 불리기

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,        # Shear angle in counter-clockwise direction as radians
    zoom_range=0.3,         # Range for random zoom. If a float
    horizontal_flip=True,   # Randomly flip inputs horizontally
    fill_mode='nearest') 

train1_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,        # Shear angle in counter-clockwise direction as radians
    zoom_range=0.3,         # Range for random zoom. If a float
    horizontal_flip=True,   # Randomly flip inputs horizontally
    fill_mode='nearest') 

train2_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,        # Shear angle in counter-clockwise direction as radians
    zoom_range=0.3,         # Range for random zoom. If a float
    horizontal_flip=True,   # Randomly flip inputs horizontally
    fill_mode='nearest') 

val_datagen = ImageDataGenerator(
    rescale=1./255,)

test_datagen = ImageDataGenerator(
    rescale=1./255,)

In [ ]:
# Numpy Array Iterator 객체 생성하여 모델 인풋으로 배치 사이즈만큼 집어넣을 준비
train_generator = train_datagen.flow(x=tr_images, y=tr_labels, batch_size=32, shuffle=True)
train1_generator = train1_datagen.flow(x=tr1_images, y=tr1_labels, batch_size=32, shuffle=True)
train2_generator = train2_datagen.flow(x=tr2_images, y=tr2_labels, batch_size=32, shuffle=True)

validation_generator = val_datagen.flow(x=val_images, y=val_labels, batch_size=32, shuffle=False)

test_generator = test_datagen.flow(x=test_images, y=test_labels, batch_size=32, shuffle=False)

### 모델 생성

In [ ]:
moat2 = moat.get_model(name='tiny_moat1_pretrain_256_no_pe_1k',input_shape=[32,32,3])
moat2 = moat._load_moat_pretrained_checkpoint(moat2, path="./model-ckpt-0")

o2 = moat2.output["stage5"]
r2 = tf.keras.layers.Flatten(name='flatten_layer1')(o2)
r2 = tf.keras.layers.BatchNormalization()(r2)
out2 = tf.keras.layers.Dense(
    units=100, activation='softmax', name='output_layer', kernel_initializer="he_normal")(r2)  

def create_model():
    return tf.keras.models.Model(moat2.input, out2)

In [ ]:
model2 = create_model()
#model2.load_weights('cifar_final_disjoint2_1.hdf5')

import warnings
warnings.filterwarnings('ignore')

opt = AdaBeliefOptimizer(learning_rate=1e-3, beta_1= 0.9, beta_2=0.999, weight_decay= 0.05,  epsilon=1e-14)

model2.compile(optimizer=opt,
          loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())

checkpointer = ModelCheckpoint(filepath='cifar_final_disjoint2_1.hdf5', verbose=1, monitor ="val_sparse_categorical_accuracy",save_best_only=True)
csv_logger = CSVLogger('cifar_final_disjoint2_1.log')

history = model2.fit(train2_generator,
                steps_per_epoch = nb_train2_samples // batch_size,      
                validation_data=validation_generator,
                validation_steps=nb_validation_samples // batch_size,  
                epochs=80,                                             
                verbose=1,
                callbacks=[csv_logger, checkpointer])

In [ ]:
model2 = create_model()
model2.load_weights('cifar_final_disjoint2_1.hdf5')

import warnings
warnings.filterwarnings('ignore')

opt = AdaBeliefOptimizer(learning_rate=1e-3, beta_1= 0.9, beta_2=0.999, weight_decay= 0.05,  epsilon=1e-14)

model2.compile(optimizer=opt,
          loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())

checkpointer = ModelCheckpoint(filepath='cifar_final_disjoint2_2.hdf5', verbose=1, monitor ="val_sparse_categorical_accuracy",save_best_only=True)
csv_logger = CSVLogger('cifar_final_disjoint2_2.log')

history = model2.fit(train2_generator,
                steps_per_epoch = nb_train2_samples // batch_size,      
                validation_data=validation_generator,
                validation_steps=nb_validation_samples // batch_size,  
                epochs=80,                                             
                verbose=1,
                callbacks=[csv_logger, checkpointer])